In [1]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("Set2")

metrics_dir = "../artifacts"
models = ["log", "rf", "xgb"]  # later we’ll add "dl" and "quantum"


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
results = []
for model_type in models:
    path = f"{metrics_dir}/{model_type}_metrics.joblib"
    if os.path.exists(path):
        data = joblib.load(path)
        test = data["test"]
        test["model"] = model_type.upper()
        results.append(test)

df = pd.DataFrame(results)
df = df[["model", "accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]]
df = df.sort_values(by="roc_auc", ascending=False).reset_index(drop=True)
df


In [ ]:
plt.figure(figsize=(10,6))
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]
for metric in metrics:
    sns.barplot(x="model", y=metric, data=df)
    plt.title(f"Model Comparison: {metric.upper()}")
    plt.ylim(0.8, 1.01)
    plt.show()


In [ ]:
import numpy as np

def plot_radar(df):
    labels = ["accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]
    num_vars = len(labels)
    angles = np.linspace(0, 2*np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(polar=True))
    
    for _, row in df.iterrows():
        values = [row[l] for l in labels]
        values += values[:1]
        ax.plot(angles, values, label=row["model"])
        ax.fill(angles, values, alpha=0.1)
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_ylim(0.8, 1.0)
    plt.title("Model Performance Radar Plot")
    plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
    plt.show()

plot_radar(df)


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

plot_dir = "../results"

fig, axes = plt.subplots(1, len(models), figsize=(5*len(models), 4))
if len(models) == 1:
    axes = [axes]

for i, model_type in enumerate(models):
    img_path = f"{plot_dir}/{model_type}_plots/confusion_matrix.png"
    if os.path.exists(img_path):
        img = mpimg.imread(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(model_type.upper())

plt.suptitle("Confusion Matrices of All Models", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
best_model = df.loc[df["roc_auc"].idxmax(), "model"]

print(" Summary:")
print(df.to_string(index=False))
print(f"\n Best Performing Model: {best_model}")

print("""
 Observations:
1. XGBoost achieved the highest ROC-AUC and PR-AUC scores, indicating superior fraud detection performance.
2. Random Forest performed nearly as well and is simpler to interpret.
3. Logistic Regression achieved high accuracy but slightly lower recall on minority (fraud) class.
4. Deep Learning and Quantum models (to be added next) will be compared similarly.
""")
